In [102]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [103]:
words = open("names.txt", 'r').read().splitlines()

In [104]:
#build the vacabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [105]:
#build dataset

block_size = 3  #how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y=torch.tensor(Y)

In [123]:
#build dataset
def build_dataset(words):
    
    block_size = 3  #how many characters do we take to predict the next one?
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y=torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


In [115]:
g = torch.Generator().manual_seed(214783647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100, generator = g)
W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn(27, generator = g)
paramters = [C, W1, b1, W2, b2]

In [116]:
sum(p.nelement() for p in paramters)

3481

In [117]:
for p in paramters:
    p.requires_grad = True

In [118]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre


In [125]:
lri=[]
lossi=[]
for i in range(10000):
    #mini batch construct
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    #forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,6)@W1 + b1)
    logits = h @ W2 + b2
    counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims = True)
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())
    # backward pass
    for p in paramters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1
    for p in paramters:
        p.data += -lr * p.grad
    

print(loss.item())

2.455749273300171
